# Conformal Prediction

The sparsity of observed instances gravely influences the degree of confidence a model prediction could have. Conformal prediction is an applicative technique that adds calibration process on top of any models to quantify uncertainties in data for both point estimators and prediction intervals. 

While there are different ways to carry out conformal prediction, NeuralProphet adopts the split conformal prediction, which requires a holdout or calibration set. To carry out split conformal prediction, the dataset has to be split into three distinct sets for training, calibration and testing respectively. An initial prediction interval is created with the base quantile regression model trained by the training dataset. Target variables in the calibration set are being compared to the predicted value, which is denoted as the [quantified uncertainty](https://towardsdatascience.com/conformal-prediction-4775e78b47b6). The final conformal prediction interval is then formed by adding the quantified uncertainty to both tails of the predicted value.

You can select *Naive* (or absolute residual) or *Conformalized Quantile Regression* (or CQR) for the conformal prediction in NeuralProphet. Here, we illustrate and further elaborate on the conformal prediction naive and CQR module using the hospital electric load dataset. The dataset has recorded the electricity consumption of a hospital in SF in 2015 by hour.

In [1]:
# much faster using the following code, but may not have the latest upgrades/bugfixes
# !pip install neuralprophet

if "google.colab" in str(get_ipython()):
    !pip install git+https://github.com/ourownstory/neural_prophet.git  # may take a while

In [2]:
import numpy as np
import pandas as pd
from neuralprophet import NeuralProphet, uncertainty_evaluate, set_log_level, set_random_seed

data_location = "http://raw.githubusercontent.com/ourownstory/neuralprophet-data/main/datasets/"
df = pd.read_csv(data_location + "energy/SF_hospital_load.csv")

/Users/leoniewagner/Documents/Uni/Stanford/dev_code/neural_prophet/.venv/lib/python3.9/site-packages/plotly_resampler/aggregation/aggregators.py:25: UserWarning: Could not import lttbc; will use a (slower) python alternative.
  warnings.warn("Could not import lttbc; will use a (slower) python alternative.")


### Data splitting

At least three subsets (i.e. testing, calibration and testing) are needed in the conformal prediction feature in NeuralProphet. You may choose to opt in a validation subset in this model. If you want to add in a validation subset to train the base model, please make sure the period of the validation subset must be in between of the training and calibration subsets. In NeuralProphet, there is a data splitting function which divide a dataset input into two subsets. You can configure the function by indicating the time series frequencies and splitting ratio. List of frequency aliases can be found <a href= 'https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases'>here</a>.

In our hospital electric load dataset, we will first divide the original dataset into training and testing set with a train-test ratio of $1/16$ and then further divide the calibation set from the training set with a train-calibration ratio of $1/11$.

In [3]:
# Create NeuralProphet object
m = NeuralProphet()

# Data spliting function split one timeseries dataframe into two
# Configure the hourly frequency by assigning 'H' to parameter freq
# Configure the splitting ratio with a value between 0 and 1 for valid_p
train_df, test_df = m.split_df(df, freq="H", valid_p=1.0 / 16)
train_df, cal_df = m.split_df(train_df, freq="H", valid_p=1.0 / 11)

INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.989% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.988% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


After splitting, we will have 7467, 746 and 547 instances extracted in sequence as training, calibration and testing set respectively.

In [4]:
train_df.shape, cal_df.shape, test_df.shape

((7467, 2), (746, 2), (547, 2))

In [5]:
print(f"Train set time range:        {train_df['ds'].min()} - {train_df['ds'].max()}")
print(f"Calibration set time range:  {cal_df['ds'].min()} - {cal_df['ds'].max()}")
print(f"Test set time range:         {test_df['ds'].min()} - {test_df['ds'].max()}")

Train set time range:        2015-01-01 01:00:00 - 2015-11-08 03:00:00
Calibration set time range:  2015-11-08 04:00:00 - 2015-12-09 05:00:00
Test set time range:         2015-12-09 06:00:00 - 2016-01-01 00:00:00


### Base model training
You can build any NeuralProphet model you deemed as fit as the base model. The calibration process in conformal prediction would be later added on the base model to quantify the uncertainty in our final estimation.

Back to our example, assuming we want to build an autoregression NeuralProphet model which can be providing us a prediction interval with 90% of confidence level. Thus we create a three-quantiles model that outputs the 5th, 50th and 95th percentile values. You can refer to the [NeuralProphet](https://neuralprophet.com/tutorials/tutorials.html), [Autoregression](https://neuralprophet.com/notebooks/autoregression_yosemite_temps.html) and [Uncertainty Estimation with Quantile Regression](https://github.com/ourownstory/neural_prophet/blob/main/tutorials/feature-use/uncertainty_quantile_regression.ipynb)  standalone tutorial sessions for the logic and applications of the features. We will be comparing the conformal prediction results between a simple and a complex 4-layer autoregression model in our illustration here.

In [6]:
# Parameter for autoregression
# Predict the value in the next hours based on the last three daysin an-hour steps
n_lags = 3 * 24

# Parameter for quantile regression
confidence_lv = 0.9
quantile_list = [round(((1 - confidence_lv) / 2), 2), confidence_lv + (1 - confidence_lv) / 2]

In [7]:
# Create a simple quantile regression model
m1 = NeuralProphet(quantiles=quantile_list)

# Create a 4-layer autoregression model as the base
m2 = NeuralProphet(
    growth="off",
    yearly_seasonality=False,
    weekly_seasonality=False,
    daily_seasonality=False,
    n_lags=n_lags,
    num_hidden_layers=4,
    d_hidden=32,
    learning_rate=0.003,
    quantiles=quantile_list,
)

After finish configuring the model, we fit the model with the train set. 

If you have further split the training dataset into training and validation, you can either (i) concatenate the two datasets in one dataset for training or (ii) assign the training and validation datasets as two separated parameters. You can know more about the validation procedure in the [Phase Train, Validate and Test procedure](https://neuralprophet.com/notebooks/test_and_crossvalidate.html#3.-Advanced:-3-Phase-Train,-Validate-and-Test-procedure).

In [8]:
%%time
# Feed the training subset in the configured NeuralProphet models
# Configure the hourly frequency by assigning 'H' to parameter freq
set_random_seed(0)
metrics1 = m1.fit(train_df, freq="H")
set_random_seed(0)
metrics2 = m2.fit(train_df, freq="H")

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.987% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 111
WARNING - (NP.config.set_lr_finder_args) - Learning rate finder: The number of batches (234) is too small than the required number for the learning rate finder (246). The results might not be optimal.


Finding best initial lr:   0%|          | 0/246 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.987% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 111


Training: 0it [00:00, ?it/s]

CPU times: user 15min 33s, sys: 19.7 s, total: 15min 52s
Wall time: 5min 59s


We use the fitted base model to forecast both the point prediction and the quantile regression prediction intervals for the testing dataset. 

In [9]:
# Perform estimation for the testing data with the trained model
forecast1 = m1.predict(test_df)[n_lags:]
forecast2 = m2.predict(test_df)[n_lags:]

INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.817% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.817% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H


Predicting: 234it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.817% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.818% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H


Predicting: 232it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


We can then plot the predicted intervals (5th, 50th and 95th percentile values) to compare the performance of the models. With the same quantile parameter, the complex model has a much narrower prediction interval comparing to the simple model.

In [10]:
# Date range shown in the plots (optional)
cutoff = 7 * 24
plotting_backend = "plotly"

# Plot the estimated value predicted with the base model
m1.highlight_nth_step_ahead_of_each_forecast(1).plot(forecast1[-cutoff:], plotting_backend=plotting_backend)

WARNING - (NP.forecaster.plot) - highlight_forecast_step_n is ignored since auto-regression not enabled.


In [11]:
m2.highlight_nth_step_ahead_of_each_forecast(1).plot(forecast2[-cutoff:], plotting_backend=plotting_backend)

### Conformal Prediction
#### Option 1: Naive Conformal Prediction

After training the base model, we then carry out the calibration process using the `naive` module. The steps are outlined as follow:
<br>i. predict the output value of the instances within the calibration set;
<br>ii. calculate absolute residual by comparing the actual and predicted value for each observation in the calibration set;
<br>iii. sort all absolute residual in ascending order; and
<br>iv. finds the quantified uncertainty ($\hat{q}$) with the desired confidence level.
<br>v. use the quantified uncertainty ($\hat{q}$) to make the final prediciton intervals

Going back to our example, we need to denote the parameter value for the calibration set and the significant level (alpha) for conformal prediction on top of the pre-trained models above.

In [12]:
# Parameter for naive conformal prediction
method = "naive"
alpha = 1 - confidence_lv

# Enable conformal predict on the pre-trained models
# Evaluate parameter is optional, refer to the "Evaluate Performance" session below
naive_forecast1 = m1.conformal_predict(
    test_df, calibration_df=cal_df, alpha=alpha, method=method, plotting_backend=plotting_backend
)
naive_forecast2 = m2.conformal_predict(
    test_df, calibration_df=cal_df, alpha=alpha, method=method, plotting_backend=plotting_backend
)

INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.866% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.866% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H


Predicting: 234it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.817% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.817% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H


Predicting: 234it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.866% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.866% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H


Predicting: 232it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.817% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.818% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H


Predicting: 232it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


The plots above demonstrate how quantified uncertainty ($\hat{q}$) change with different confidence level (`1-alpha`).  In the Naive conformal prediction, uncertainty is quantified using the absolute residual . We calculate the  absolute residual for each instance within the calibration dataset and arrange the residual in an ascending order. The uncertainty ($\hat{q}$) is quantified by locating the absolute residual  at your desired confidence level. We will then add the uncertainty value ($\hat{q}$) to both sides of the prediction interval generated in the base model.

The method conformal_predict gives back the forecast with the updated uncertainty intervals, which is enough for most use cases. If you want to acess the $\hat{q}$ values for a more detailed analysis, you can set the optional parameter `show_all_PI` to true. 

This will give the $\hat{q}$ values for each model. As they are all positive, you will see the naive interval extending beyond that of simple QR. The weaker model has a large gap between these two intervals since it has a larger $\hat{q}$ value, while for complex model, the bounds are shifted much less.

In [13]:
naive_forecast1 = m1.conformal_predict(
    test_df,
    calibration_df=cal_df,
    alpha=alpha,
    method=method,
    show_all_PI=True,
)
naive_forecast2 = m2.conformal_predict(
    test_df,
    calibration_df=cal_df,
    alpha=alpha,
    method=method,
    show_all_PI=True,
)

INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.866% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.866% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H


Predicting: 234it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.817% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.817% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H


Predicting: 234it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.866% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.866% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H


Predicting: 232it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.817% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.818% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H


Predicting: 232it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


In [15]:
# get q_hat
naive_qhat1 = (naive_forecast1["yhat1"] - naive_forecast1["yhat1 5.0%"]).dropna()
naive_qhat2 = (naive_forecast2["yhat1"] - naive_forecast2["yhat1 5.0%"]).dropna()

naive_qhat1.unique()[0], naive_qhat2.unique()[0]

(79.181396484375, 13.8187255859375)

We plot the conformal prediction intervals again below. For the weaker model, the conformal prediction intervals has a wider width as such it can capture more more actual values than the quantile regression prediction intervals (in blue). The complex model in contrast has a much narrower prediction interval comparing to the simple model.

In [16]:
m1.conformal_plot(naive_forecast1[-cutoff:], plotting_backend="plotly")

WARNING - (NP.forecaster.plot) - highlight_forecast_step_n is ignored since auto-regression not enabled.


In [17]:
m2.conformal_plot(naive_forecast2[-cutoff:], plotting_backend="plotly")

#### Option 2: Conformalized Quantile Regression

In [Conformalized Quantile Regression](https://arxiv.org/abs/1905.03222), or the `cqr` module, the method runs as follows:
<br>i. A sorted list of non-conformity scores is calculated as the differences between data points from the calibration dataset and their nearest prediction quantile, which provides a measure of how well the data fits the current quantile regression model.
<br>ii. The differences are calculated such they are negative for data points within the quantile regression interval and positive if they are outside the interval.
<br>iii. The quantiles from the regression model are adjusted by an amount that satisfies the desired error rate, which is the portion of data points lying outside the interval, based on the calibration data's non-conformity scores.

In [18]:
# Parameter for Conformalized Quantile Regression
method = "cqr"

# Enable conformal predict on the pre-trained models
# Evaluate parameter is optional, refer to the "Evaluate Performance" session below
cqr_forecast1 = m1.conformal_predict(
    test_df, calibration_df=cal_df, alpha=alpha, method=method, plotting_backend=plotting_backend, show_all_PI=True
)
cqr_forecast2 = m2.conformal_predict(
    test_df, calibration_df=cal_df, alpha=alpha, method=method, plotting_backend=plotting_backend, show_all_PI=True
)

INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.866% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.866% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H


Predicting: 234it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.817% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.817% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H


Predicting: 234it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.866% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.866% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H


Predicting: 232it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.817% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.818% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H


Predicting: 232it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


The plots above show how the prediction interval bounds' adjustment, which is denoted $\hat{q}$, is calculated from the non-conformity scores of the calibration data and each quantile regression model m1 and m2. Given an error rate alpha, the sorted list of non-conformity scores yields a value of $\hat{q}$ such that the portion of scores in the list greater than $\hat{q}$ is equal to that error rate. What this means is that we update the prediction interval bounds based on the calibration data so that we are more reasonably confident that the training set data satisfies a similar error rate as alpha when compared to the new quantile regression model. There are two scenarios for what the CQR model means based on the value of $\hat{q}$.
- If the one-sided prediction interval width adjustment is **positive**, then CQR **extends** beyond the QR intervals, as it deems the QR interval to be too **confident**.
- Conversely, if the one-sided prediction interval width adjustment is **negative**, then CQR **contracts** the QR intervals, as it deems the QR interval to be too **conservative**.

In the above figure, you can see the $\hat{q}$ values for each model. As they are all positive, you will see the CQR interval extending beyond that of simple QR. Model m1 has a large gap between these two intervals since it has a larger $\hat{q}$ value, while for m2, the bounds are shifted much less.

Plot *QR* forecasts and *CQR* conformal prediction interval with OOS test_df

In [19]:
m1.conformal_plot(cqr_forecast1[-cutoff:], plotting_backend="plotly")

WARNING - (NP.forecaster.plot) - highlight_forecast_step_n is ignored since auto-regression not enabled.


In [20]:
m2.conformal_plot(cqr_forecast2[-cutoff:], plotting_backend="plotly")

### Evaluate Performance

We are using interval width and miscoverage rate as the performance metrics.
- `interval_width`: The average prediction interval, or q_hat multiplied by 2 because it is static or non-adaptive, this is also knowns as the *efficiency* metric.
- `miscoverage_rate`: The actual miscoverage error rate on the OOS test set, this is also knowns as the *validity* metric.

The smaller the metrics are, the better the models are performing.

Let's test with the models we trained above. We aggregate the performance metrics for quantile regression, naive conformal prediction and conformalized quantile regression. The performance metrics are calculated when you have turned on the `evaluate` parameter in the conformal prediction feature.

In [22]:
# Quantile regression evaluation
qr_eval1 = uncertainty_evaluate(forecast1)
qr_eval2 = uncertainty_evaluate(forecast2)
qr_eval = pd.concat([qr_eval1, qr_eval2]).reset_index(drop=True)

# Naive conformal prediction evaluation
naive_eval1 = uncertainty_evaluate(naive_forecast1)
naive_eval2 = uncertainty_evaluate(naive_forecast2)
naive_eval = pd.concat([naive_eval1, naive_eval2]).reset_index(drop=True)

# Conformalized quantile regression evaluation
cqr_eval1 = uncertainty_evaluate(cqr_forecast1)
cqr_eval2 = uncertainty_evaluate(cqr_forecast2)
cqr_eval = pd.concat([cqr_eval1, cqr_eval2]).reset_index(drop=True)

# concatenate the evaluation results whith 1 in the first row and 2 in the second row
models = ["m1", "m2"]
eval_df = pd.concat(
    [qr_eval, naive_eval, cqr_eval], axis=1, keys=["qr", "naive", "cqr"], names=["method", "step", "metric"]
)
eval_df

method             qr                           naive                   \
step            yhat1                           yhat1                    
metric interval_width miscoverage_rate interval_width miscoverage_rate   
0          154.320040         0.498947     321.346050         0.107861   
1           18.022723         0.364211      47.769659         0.050526   

method            cqr                   
step            yhat1                   
metric interval_width miscoverage_rate  
0          309.400519         0.107861  
1           46.696792         0.052632

**Performance of the trained model above**:

- This notebook is only using single forecast timestep models, hence we would only have `yhat1` as the point estimator.
- Both conformal prediction methods (*Naive* and *CQR*), significantly outperforms vanilla *QR* in terms of `miscoverage_rate`. This shows that the *QR* is **too overconfident** in its quantile values. Hence the `interval_width` needs to be **broadened further** in order for its actual `miscoverage_rate` (on out-of-sample test set) to converge to the specified `alpha` at `0.1` (from calibration set).
- The `interval_width` doubles the quantified uncertainties (`qhat1`) for both the *Naive* conformal predictions, as such you will be seeing symmetrical prediction intervals.
- The complex model (`m2`) has a lower `interval_width` and `miscoverage_rate` than the simple model (`m1`), acroos all uncertainty methods. The simpler model (`m1`) have `miscoverage_rate` bigger than the confidence level (i.e., 90% or `1-alpha`) while the complex autoregressioin model (`m2`) have a `miscoverage_rate` smaller than the confidence level. As such, we can conclude that **the more complex the model is, the better the data are fitted and the more accurate the prediction are made**.
- When we are only looking at the simple quantile regression model (`m1`), *CQR* is more preferable as it has narrower prediction `interval_width` yet similar `miscoverage_rate` than *Naive*.
- As for the complex model (`m2`), *Naive* has a slightly better `interval_width`, but *CQR* has a slightly better `miscoverage_rate`. You may want to feed the model with more data to determine which method more preferable.
